# Librerías implementadas

In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.5 MB/s eta 0:00:00


In [2]:
import random
import faker
from datetime import datetime

# Generación BIG DATA

## Estableciendo cantidades de registros

In [3]:
fake = faker.Faker()

cantidad_empresas = 1000
cantidad_telefonos = int(cantidad_empresas * 1.2)
cantidad_ciudades = 20
cantidad_sucursales = 100
cantidad_empresas_sucursal = int(cantidad_sucursales * 0.75)
cantidad_sectores = 500
cantidad_clientes = 10000
cantidad_empleados = 2000
cantidad_marcas = 100
cantidad_productos = 10000
cantidad_descuento = 6
cantidad_usuarios = 300
cantidad_personas = cantidad_usuarios + cantidad_clientes + cantidad_empleados
cantidad_ventas = 30000
cantidad_ciudad = 100
cantidad_contratos = cantidad_empleados + cantidad_empresas
cantidad_transacciones = cantidad_ventas


## Funciones de escritura

In [4]:
def elementos_lista(lista):
    valor = ""
    for i in range(len(lista)):
        if i < len(lista) - 1:
            valor += lista[i].replace('\n', ' ').replace('\r', ' ') + ", "
        else:
            valor += lista[i]
    return valor


def generar_values(data, file):
    cadena_data = f"({elementos_lista(data)});\nGO\n"
    file.write(cadena_data)


def generar_cabecera(esquema, tabla, parametros, file):
    cadena_parametros = elementos_lista(parametros)
    insert_data = f"\nINSERT INTO [{esquema}].{tabla}({cadena_parametros})\nVALUES "
    file.write(insert_data)

## Implementando Entidades

In [5]:
class Persona:
    esquema = "adm"
    tabla = "persona"
    campos = ["dni", "nombre", "apellido_paterno", "apellido_materno", "fecha_nacimiento"]

    @staticmethod
    def generar_insert_persona(file, cantidad):
        d = []
        for i in range(1, cantidad + 1):
            generar_cabecera(Persona.esquema, Persona.tabla, Persona.campos, file)
            dni = None
            while True:
              dni = random.randint(10000000, 99999999)
              if dni not in d:
                  d.append(dni)
                  break
            dni = f"'{dni}'"
            nombre = f"'{fake.first_name()}'"
            apellido_paterno = f"'{fake.last_name()}'"
            apellido_materno = f"'{fake.last_name()}'"
            fecha_nacimiento = f"""'{fake.date_of_birth(
                minimum_age=18,
                maximum_age=80
                ).strftime('%Y-%m-%d')}'"""

            generar_values([dni, nombre, apellido_paterno, apellido_materno,
                            fecha_nacimiento], file)


class FormaPago:
    esquema = "ven"
    tabla = "forma_pago"
    campos = ["forma"]
    lista = [["'efectivo'"], ["'tarjeta'"]]

    @staticmethod
    def generar_insert_forma_pago(file):
        for i in range(len(FormaPago.lista)):
            generar_cabecera(FormaPago.esquema, FormaPago.tabla, FormaPago.campos, file)
            generar_values(FormaPago.lista[i], file)


class MotivoContrato:
    esquema = "adm"
    tabla = "motivo_contrato"
    campos = ["motivo"]
    lista = [["'trabajo'"], ["'practica'"], ["'temporal'"], ["'servicio externo'"]]

    @staticmethod
    def generar_insert_motivo_contrato(file):
        for i in range(len(MotivoContrato.lista)):
            generar_cabecera(MotivoContrato.esquema, MotivoContrato.tabla, MotivoContrato.campos, file)
            generar_values(MotivoContrato.lista[i], file)


class Contrato:
    esquema = "adm"
    tabla = "contrato"
    campos = ["fecha_inicio", "fecha_finalizacion", "descripcion", "monto",
              "id_forma_pago", "id_motivo_contrato"]

    @staticmethod
    def generar_insert_contrato(file, cantidad):
        for i in range(1, cantidad + 1):
            generar_cabecera(Contrato.esquema, Contrato.tabla, Contrato.campos, file)

            fecha_inicio = f"""'{fake.date_between(
                start_date='-10y',
                end_date='today'
                ).strftime('%Y-%m-%d')}'"""
            fecha_finalizacion = f"""'{fake.date_between(
                start_date='today',
                end_date='+10y'
                ).strftime('%Y-%m-%d')}'"""
            descripcion = f"'{fake.sentence()}'"
            monto = random.uniform(1500, 5000)
            monto = str("{:.2f}".format(monto))
            id_forma_pago = str(random.randint(1, len(FormaPago.lista)))
            id_motivo_contrato = str(random.randint(1, len(MotivoContrato.lista)))

            generar_values([fecha_inicio, fecha_finalizacion, descripcion,
                            monto, id_forma_pago, id_motivo_contrato], file)


class EmpresasAsociadas:
    esquema = "adm"
    tabla = "empresa_asociada"
    campos = ["ruc", "nombre", "direccion", "id_contrato"]

    @staticmethod
    def generar_insert_empresas_asociadas(file, cantidad):
        dir = []
        r = []
        for i in range(1, cantidad + 1):
            generar_cabecera(EmpresasAsociadas.esquema, EmpresasAsociadas.tabla,
                             EmpresasAsociadas.campos, file)
            ruc = None
            while True:
              ruc = random.randint(10000000000, 99999999999)
              if ruc not in r:
                  r.append(ruc)
                  break
            ruc = f"'{ruc}'"

            nombre = f"'{fake.company()}'"

            direccion = None
            while True:
              direccion = fake.address()
              if direccion not in dir:
                  dir.append(direccion)
                  break
            direccion = f"'{direccion}'"

            id_contrato = str(i)
            generar_values([ruc, nombre, direccion, id_contrato], file)


class Telefono:
    esquema = "adm"
    tabla = "telefono"
    campos = ["prefijo", "telefono", "id_empresa_asociada"]

    @staticmethod
    def generar_insert_telefono(file, cantidad):
        tel = []
        for i in range(1, cantidad + 1):
            generar_cabecera(Telefono.esquema, Telefono.tabla, Telefono.campos, file)
            prefijo = f"'{random.randint(1, 999)}'"

            telefono = None
            while True:
              telefono = random.randint(900000000, 999999999)
              if telefono not in tel:
                  tel.append(telefono)
                  break

            telefono = f"'{telefono}'"
            id_empresas_asociadas = str(random.randint(1, cantidad_empresas))

            generar_values([prefijo, telefono, id_empresas_asociadas], file)


class Ciudad :
    esquema = "adm"
    tabla = "ciudad"
    campos = ["nombre"]

    @staticmethod
    def generar_insert_ciudad(file, cantidad):
        city = []
        for i in range(1, cantidad + 1):
            generar_cabecera(Ciudad.esquema, Ciudad.tabla, Ciudad.campos, file)
            nombre = None
            while True:
              nombre = fake.city()
              if nombre not in city:
                  city.append(nombre)
                  break
            nombre = f"'{nombre}'"
            generar_values([nombre], file)


class Sucursal:
    esquema = "adm"
    tabla = "sucursal"
    campos = ["nombre", "direccion", "id_ciudad"]

    @staticmethod
    def generar_insert_sucursal(file, cantidad):
        dir = []
        for i in range(1, cantidad + 1):
            generar_cabecera(Sucursal.esquema, Sucursal.tabla, Sucursal.campos, file)
            nombre = f"'{fake.company()}'"
            direccion = None
            while True:
              direccion = fake.address()
              if direccion not in dir:
                  dir.append(direccion)
                  break
            direccion = f"'{direccion}'"
            id_ciudad = str(random.randint(1, cantidad))
            generar_values([nombre, direccion, id_ciudad], file)


class EmpresasSucursal:
    esquema = "adm"
    tabla = "empresa_sucursal"
    campos = ["id_empresa", "id_sucursal"]

    @staticmethod
    def generar_insert_empresas_sucursal(file, cant, cant_empresa, cant_sucursal):
        conjunto = []
        for i in range(1, cant + 1):
            generar_cabecera(EmpresasSucursal.esquema, EmpresasSucursal.tabla, EmpresasSucursal.campos, file)
            id_empresa = None
            id_sucursal = None

            while True:
              id_empresa = str(random.randint(1, cant_empresa))
              id_sucursal = str(random.randint(1, cant_sucursal))
              pk = {id_sucursal, id_empresa}

              if pk not in conjunto:
                  conjunto.append(pk)
                  break

            generar_values([id_empresa, id_sucursal], file)


class TipoSector:
    esquema = "adm"
    tabla = "tipo_sector"
    campos = ["tipo"]
    lista = [["'comestibles'"], ["'tecnologia'"], ["'bebidas'"],
     ["'vestimenta'"], ["'cosmeticos'"], ["'higiene'"], ["'dermatologia'"],
      ["'farmaceutica'"]]

    @staticmethod
    def generar_insert_tipo_sector(file):
        for i in range(len(TipoSector.lista)):
            generar_cabecera(TipoSector.esquema, TipoSector.tabla, TipoSector.campos, file)
            generar_values(TipoSector.lista[i], file)


class Sector:
    esquema = "adm"
    tabla = "sector"
    campos = ["nombre", "id_tipo_sector", "id_sucursal"]

    @staticmethod
    def generar_insert_sector(file, cant, cant_sucursal):
        for i in range(1, cant + 1):
            generar_cabecera(Sector.esquema, Sector.tabla, Sector.campos, file)

            nombre = f"'{fake.company()}'"
            id_tipo_sector = str(random.randint(1, len(TipoSector.lista)))
            id_sucursal = str(random.randint(1, cant_sucursal))

            generar_values([nombre, id_tipo_sector, id_sucursal], file)


class TipoEmpleado:
    esquema = "adm"
    tabla = "tipo_empleado"
    campos = ["tipo"]
    lista = [["'administrativo'"], ["'vendedor'"],
     ["'supervisor'"], ["'cajero'"], ["'repartidor'"],
      ["'limpieza'"], ["'seguridad'"], ["'administrador'"]]

    @staticmethod
    def generar_insert_tipo_empleado(file):
        for i in range(len(TipoEmpleado.lista)):
            generar_cabecera(TipoEmpleado.esquema, TipoEmpleado.tabla, TipoEmpleado.campos, file)
            generar_values(TipoEmpleado.lista[i], file)


class Empleado:
    esquema = "adm"
    tabla = "empleado"
    campos = ["codigo", "id_persona", "horas_trabajo", "id_tipo_empleado",
              "id_supervisor", "id_contrato", "id_sector"]

    @staticmethod
    def generar_insert_empleado(file, cantidad):
        id = 1
        for i in range(cantidad_empresas + 1, cantidad + 1):
            generar_cabecera(Empleado.esquema, Empleado.tabla, Empleado.campos, file)

            codigo = f"'EMP-{i}'"
            id_persona = f"{id}"
            id += 1
            horas_trabajo = str(random.randint(1, 168))
            id_tipo_empleado = str(random.randint(1, len(TipoEmpleado.lista)))
            id_supervisor = "null"
            id_contrato = str(i)
            id_sector = str(random.randint(1, cantidad_sectores))

            generar_values([codigo, id_persona, horas_trabajo, id_tipo_empleado,
                            id_supervisor, id_contrato, id_sector], file)


class TipoCliente:
    esquema = "ven"
    tabla = "tipo_cliente"
    campos = ["tipo"]
    lista = [["'nuevo'"], ["'regular'"], ["'frecuente'"]]

    @staticmethod
    def generar_insert_tipo_cliente(file):
        for i in range(len(TipoCliente.lista)):
            generar_cabecera(TipoCliente.esquema, TipoCliente.tabla, TipoCliente.campos, file)
            generar_values(TipoCliente.lista[i], file)


class Cliente:
    esquema = "ven"
    tabla = "cliente"
    campos = ["id_persona", "id_tipo_cliente"]

    @staticmethod
    def generar_insert_cliente(file, cantidad):
        id = cantidad_empleados + 1
        for i in range(1, cantidad + 1):
            generar_cabecera(Cliente.esquema, Cliente.tabla, Cliente.campos, file)
            id_persona = str(id)
            id += 1
            id_tipo_cliente = str(random.randint(1, len(TipoCliente.lista)))

            generar_values([id_persona, id_tipo_cliente], file)


class TipoProducto:
    esquema = "ven"
    tabla = "tipo_producto"
    campos = ["tipo"]
    lista = [["'bebida'"], ["'comida'"], ["'higiene'"], ["'limpieza'"],
     ["'cosmetico'"], ["'electrodomestico'"], ["'deporte'"], ["'ropa'"],
      ["'accesorio'"], ["'medicamento'"], ["'insumo'"]]

    @staticmethod
    def generar_insert_tipo_producto(file):
        for i in range(len(TipoProducto.lista)):
            generar_cabecera(TipoProducto.esquema, TipoProducto.tabla, TipoProducto.campos, file)
            generar_values(TipoProducto.lista[i], file)


class MarcaProducto:
    esquema = "ven"
    tabla = "marca_producto"
    campos = ["marca"]

    @staticmethod
    def generar_insert_marca_producto(file, cantidad):
        for i in range(1, cantidad + 1):
            generar_cabecera(MarcaProducto.esquema, MarcaProducto.tabla, MarcaProducto.campos, file)
            marca = f"'{fake.company()}'"
            generar_values([marca], file)


class UnidadMedida:
    esquema = "ven"
    tabla = "unidad_medida"
    campos = ["unidad"]
    lista = [["'kg'"], ["'g'"], ["'ml'"], ["'lt'"]]

    @staticmethod
    def generar_insert_unidad_medida(file):
        for i in range(len(UnidadMedida.lista)):
            generar_cabecera(UnidadMedida.esquema, UnidadMedida.tabla, UnidadMedida.campos, file)
            generar_values(UnidadMedida.lista[i], file)


class Oferta:
    esquema = "ven"
    tabla = "oferta"
    campos = ["motivo", "descuento"]
    lista = [["'aniversario'", "50"], ["'dia de la madre'", "25"],
     ["'dia del padre'", "17"], ["'indepencia'", "15"]]

    @staticmethod
    def generar_insert_oferta(file):
        for i in range(len(Oferta.lista)):
            generar_cabecera(Oferta.esquema, Oferta.tabla, Oferta.campos, file)
            generar_values(Oferta.lista[i], file)


class Producto:
    esquema = "ven"
    tabla = "producto"
    campos = ["nombre", "descripcion", "precio", "fecha_creacion",
              "fecha_vencimiento", "contenido_neto", "existencias",
              "id_tipo_producto", "id_marca", "id_unidad_medida",
              "id_oferta", "id_sector"]

    @staticmethod
    def generar_insert_producto(file, cantidad):
        for i in range(cantidad):
            generar_cabecera(Producto.esquema, Producto.tabla, Producto.campos, file)

            nombre = f"'{fake.word().capitalize()} {fake.word().capitalize()}'"
            descripcion = f"'{fake.sentence()}'"
            precio = str("{:.2f}".format(random.uniform(1, 500)))
            fecha_creacion = f"""'{fake.date_between(
                start_date='-10y',
                end_date='today'
                ).strftime('%Y-%m-%d')}'"""
            fecha_vencimiento = f"""'{fake.date_between(
                start_date='today',
                end_date='+1y'
                ).strftime('%Y-%m-%d')}'"""
            contenido_neto = str(random.randint(1, 300))
            existencias = str(random.randint(50, 300))
            id_tipo_producto = str(random.randint(1, len(TipoProducto.lista)))
            id_marca = str(random.randint(1, cantidad_marcas))
            id_unidad_medida = str(random.randint(1, len(UnidadMedida.lista)))
            id_oferta = str(random.randint(1, len(Oferta.lista)))
            id_sector = str(random.randint(1, cantidad_sectores))
            generar_values([nombre, descripcion, precio, fecha_creacion,
                            fecha_vencimiento, contenido_neto, existencias,
                            id_tipo_producto, id_marca, id_unidad_medida,
                            id_oferta, id_sector], file)


class EstadoVenta:
    esquema = "ven"
    tabla = "estado_venta"
    campos = ["estado"]
    lista = [["'pendiente'"], ["'pagado'"], ["'cancelado'"]]

    @staticmethod
    def generar_insert_estado_venta(file):
        for i in range(len(EstadoVenta.lista)):
            generar_cabecera(EstadoVenta.esquema, EstadoVenta.tabla, EstadoVenta.campos, file)
            generar_values(EstadoVenta.lista[i], file)


class Banco:
    esquema = "ven"
    tabla = "banco"
    campos = ["nombre"]
    lista = [["'BCP'"], ["'Interbank'"], ["'Scotiabank'"]]

    @staticmethod
    def generar_insert_banco(file):
        for i in range(len(Banco.lista)):
            generar_cabecera(Banco.esquema, Banco.tabla, Banco.campos, file)
            generar_values(Banco.lista[i], file)

class Transaccion:
    esquema = "ven"
    tabla = "transaccion"
    campos = ["tarjeta", "id_banco"]

    @staticmethod
    def generar_insert_transaccion(file, cantidad):
        for i in range(1, cantidad + 1):
            generar_cabecera(Transaccion.esquema, Transaccion.tabla, Transaccion.campos, file)
            tarjeta = f"'{random.randint(1000000000000000, 9999999999999999)}'"
            id_banco = str(random.randint(1, len(Banco.lista)))
            generar_values([tarjeta, id_banco], file)


class VentaCabecera:
    esquema = "ven"
    tabla = "venta_cabecera"
    campos = ["fecha_cancelacion", "id_transaccion", "id_estado_venta",
              "id_forma_pago", "id_cliente"]
    t = 1

    def obtener_valor(idx):
        lista = [f"{VentaCabecera.t}", "null"]
        VentaCabecera.t += 1
        return lista[idx]


    @staticmethod
    def generar_insert_venta_cabecera(file, cantidad):
        for i in range(1, cantidad + 1):
            generar_cabecera(VentaCabecera.esquema, VentaCabecera.tabla, VentaCabecera.campos, file)
            fecha_cancelacion = f"""'{fake.date_between(
                start_date='-10y',
                end_date='today'
                ).strftime('%Y-%m-%d')}'"""
            id_transaccion = str(VentaCabecera.obtener_valor(random.randint(0,1)))
            id_estado_venta = str(random.randint(1, len(EstadoVenta.lista)))
            id_forma_pago = str(random.randint(1, len(FormaPago.lista)))
            id_cliente = str(random.randint(1, cantidad_clientes))
            generar_values([fecha_cancelacion, id_transaccion, id_estado_venta,
                            id_forma_pago, id_cliente], file)


class VentaDetalle:
    esquema = "ven"
    tabla = "venta_detalle"
    campos = ["id_venta_cabecera", "id_producto"]

    @staticmethod
    def generar_insert_venta_detalle(file, cant_venta, cant_producto):
        conjunto = []
        for i in range(1, cant_venta + 1):
            for j in range(random.randint(1, 8)):
                generar_cabecera(VentaDetalle.esquema, VentaDetalle.tabla, VentaDetalle.campos, file)
                id_venta_cabecera = None
                id_producto = None

                while True:
                    id_venta_cabecera = str(i)
                    id_producto = str(random.randint(1, cant_producto))
                    pk = {id_producto, id_venta_cabecera}

                    if pk not in conjunto:
                        conjunto.append(pk)
                        break

                generar_values([id_venta_cabecera, id_producto], file)


class Rol:
    esquema = "sec"
    tabla = "rol"
    campos = ["rol"]
    lista = [["'administrador'"], ["'sistema'"], ["'auditor'"]]

    @staticmethod
    def generar_insert_rol(file):
        for i in range(len(Rol.lista)):
            generar_cabecera(Rol.esquema, Rol.tabla, Rol.campos, file)
            generar_values(Rol.lista[i], file)


class Usuario:
    esquema = "sec"
    tabla = "usuario"
    campos = ["codigo", "id_persona", "correo", "contrasena", "flag", "id_rol"]

    @staticmethod
    def generar_insert_usuario(file, cantidad):
        id = cantidad_empleados + cantidad_clientes + 2
        for i in range(3, cantidad):
            generar_cabecera(Usuario.esquema, Usuario.tabla, Usuario.campos, file)
            codigo = f"'USER-{i}'"
            id_persona = str(id)
            id += 1
            correo = f"'{fake.email()}'"
            contrasena = f"'{fake.password()}'"
            estado = str(random.randint(0, 1))
            id_rol = str(random.randint(1, len(Rol.lista)))

            generar_values([codigo, id_persona, correo, contrasena, estado, id_rol], file)

## Controlador

In [6]:
def main():
    archivo_name = "Script_BIGDATA_supermercado.sql"

    with open(archivo_name, "w") as archivo:
        archivo.write("USE bd_supermercado\nGO\n")

    archivo.close()

    file = open(archivo_name, "a")

    Persona.generar_insert_persona(file, cantidad_personas)

    # Generar INSERTS dentro de la tabla forma_pago
    FormaPago.generar_insert_forma_pago(file)

    # Generar INSERTS dentro de la tabla motivo_contrato
    MotivoContrato.generar_insert_motivo_contrato(file)

    # Generar INSERTS dentro de la tabla contrato
    Contrato.generar_insert_contrato(file, cantidad_contratos)

    # Generar INSERTS dentro de la tabla empresas_asociadas
    EmpresasAsociadas.generar_insert_empresas_asociadas(file, cantidad_empresas)

    # Generar INSERTS dentro de la tabla telefono
    Telefono.generar_insert_telefono(file, cantidad_telefonos)

    # Generar INSERTS dentro de la tabla ciudad
    Ciudad.generar_insert_ciudad(file, cantidad_ciudad)

    # Generar INSERTS dentro de la tabla sucursal
    Sucursal.generar_insert_sucursal(file, cantidad_sucursales)

    # Generar INSERTS dentro de la tabla empresas_sucursal
    EmpresasSucursal.generar_insert_empresas_sucursal(file, cantidad_empresas_sucursal, cantidad_empresas, cantidad_sucursales)

    # Generar INSERTS dentro de la tabla tipo_sector
    TipoSector.generar_insert_tipo_sector(file)

    # Generar INSERTS dentro de la tabla sector
    Sector.generar_insert_sector(file, cantidad_sectores, cantidad_sucursales)

    # Generar INSERTS dentro de la tabla tipo_empleado
    TipoEmpleado.generar_insert_tipo_empleado(file)

    # Generar INSERTS dentro de la tabla empleado
    Empleado.generar_insert_empleado(file, cantidad_empleados)

    # Generar INSERTS dentro de la tabla sector
    TipoCliente.generar_insert_tipo_cliente(file)

    # Generar INSERTS dentro de la tabla cliente
    Cliente.generar_insert_cliente(file, cantidad_clientes)

    # Generar INSERTS dentro de la tabla tipo_producto
    TipoProducto.generar_insert_tipo_producto(file)

    # Generar INSERTS dentro de la tabla marca_producto
    MarcaProducto.generar_insert_marca_producto(file, cantidad_marcas)

    # Generar INSERTS dentro de la tabla unidad_medida
    UnidadMedida.generar_insert_unidad_medida(file)

    # Generar INSERTS dentro de la tabla descuento
    Oferta.generar_insert_oferta(file)

    # Generar INSERTS dentro de la tabla producto
    Producto.generar_insert_producto(file, cantidad_productos)

    EstadoVenta.generar_insert_estado_venta(file)

    Banco.generar_insert_banco(file)

    Transaccion.generar_insert_transaccion(file, cantidad_transacciones)

    # Generar INSERTS dentro de la tabla venta_cabecera
    VentaCabecera.generar_insert_venta_cabecera(file, cantidad_ventas)

    # Generar INSERTS dentro de la tabla venta_detalle
    VentaDetalle.generar_insert_venta_detalle(file, cantidad_ventas, cantidad_productos)

    # Generar INSERTS dentro de la tabla rol
    Rol.generar_insert_rol(file)

    # Generar INSERTS dentro de la tabla usuario
    Usuario.generar_insert_usuario(file, cantidad_usuarios)

    file.close()

if __name__ == "__main__":
    main()